In [ ]:
# Gaussian Blur Demo for Computer Vision Course
# Simple implementation for students with 3 weeks of Python experience

import math
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

ModuleNotFoundError: No module named 'Intro_To_AI'

In [1]:
def create_gaussian_kernel(size, sigma):
    """
    Create a Gaussian blur kernel (filter)
    
    Parameters:
    - size: kernel size (must be odd number, e.g., 3, 5, 7)
    - sigma: standard deviation (controls blur strength)
    
    Returns:
    - 2D list representing the Gaussian kernel
    """
    print(f"Creating Gaussian kernel: size={size}x{size}, sigma={sigma}")
    
    # Make sure size is odd
    if size % 2 == 0:
        size += 1
        print(f"Adjusted kernel size to {size} (must be odd)")
    
    # Calculate the center of the kernel
    center = size // 2
    
    # Initialize empty kernel
    kernel = []
    kernel_sum = 0  # We'll use this to normalize the kernel
    
    # Generate Gaussian values
    for y in range(size):
        row = []
        for x in range(size):
            # Distance from center
            dx = x - center
            dy = y - center
            
            # Gaussian formula: e^(-(x²+y²)/(2σ²))
            distance_squared = dx * dx + dy * dy
            gaussian_value = math.exp(-distance_squared / (2 * sigma * sigma))
            
            row.append(gaussian_value)
            kernel_sum += gaussian_value
        
        kernel.append(row)
    
    # Normalize the kernel (make sure all values sum to 1)
    print("Normalizing kernel...")
    for y in range(size):
        for x in range(size):
            kernel[y][x] = kernel[y][x] / kernel_sum
    
    return kernel

In [2]:
def print_kernel(kernel):
    """Print the kernel values in a nice format"""
    print("Gaussian Kernel:")
    for row in kernel:
        formatted_row = [f"{value:.4f}" for value in row]
        print("  " + " ".join(formatted_row))
    print()

In [3]:
def apply_convolution(image_array, kernel):
    """
    Apply convolution (blur) to an image using the kernel
    
    Parameters:
    - image_array: 2D array representing grayscale image
    - kernel: 2D array representing the blur kernel
    
    Returns:
    - blurred_image: 2D array with blur applied
    """
    print("Applying convolution...")
    
    # Get dimensions
    img_height, img_width = len(image_array), len(image_array[0])
    kernel_size = len(kernel)
    kernel_center = kernel_size // 2
    
    # Create output image (same size as input)
    blurred_image = []
    
    # Process each pixel
    for y in range(img_height):
        row = []
        for x in range(img_width):
            
            # Calculate weighted sum for this pixel
            weighted_sum = 0
            
            # Apply kernel around current pixel
            for ky in range(kernel_size):
                for kx in range(kernel_size):
                    # Calculate image coordinates
                    img_y = y + ky - kernel_center
                    img_x = x + kx - kernel_center
                    
                    # Handle edges by clamping to image boundaries
                    img_y = max(0, min(img_height - 1, img_y))
                    img_x = max(0, min(img_width - 1, img_x))
                    
                    # Multiply pixel value by kernel weight
                    pixel_value = image_array[img_y][img_x]
                    kernel_weight = kernel[ky][kx]
                    weighted_sum += pixel_value * kernel_weight
            
            row.append(int(weighted_sum))
        
        blurred_image.append(row)
        
        # Show progress (every 50 rows)
        if (y + 1) % 50 == 0:
            print(f"  Processed {y + 1}/{img_height} rows")
    
    return blurred_image

In [4]:
def load_and_prepare_image(filename):
    """Load image and convert to grayscale for processing"""
    print(f"Loading image: {filename}")
    
    try:
        # Load image using PIL
        img = Image.open(filename)
        print(f"Original image size: {img.size}")
        
        # Convert to grayscale for simplicity
        img_gray = img.convert('L')
        
        # Convert to 2D list of pixel values
        img_array = []
        for y in range(img_gray.height):
            row = []
            for x in range(img_gray.width):
                pixel_value = img_gray.getpixel((x, y))
                row.append(pixel_value)
            img_array.append(row)
        
        return img_array
        
    except FileNotFoundError:
        print(f"Error: Could not find image file '{filename}'")
        print("Please make sure 'test.jpg' is in the same directory as this script.")
        return None

In [5]:
def save_image_array(image_array, filename):
    """Save 2D array as image file"""
    height = len(image_array)
    width = len(image_array[0])
    
    # Create PIL image from array
    img = Image.new('L', (width, height))
    
    for y in range(height):
        for x in range(width):
            img.putpixel((x, y), image_array[y][x])
    
    img.save(filename)
    print(f"Saved result as: {filename}")

In [6]:
def demonstrate_gaussian_blur():
    """Main demonstration function"""
    print("=== Gaussian Blur Demonstration ===")
    print("This demo shows how computer vision applies blur effects to images")
    print()
    
    # Load the test image
    original_image = load_and_prepare_image("test.jpg")
    if original_image is None:
        return
    
    # Test different blur strengths
    blur_configs = [
        {"size": 3, "sigma": 1.0, "name": "Light Blur"},
        {"size": 5, "sigma": 1.5, "name": "Medium Blur"},
        {"size": 7, "sigma": 2.0, "name": "Heavy Blur"},
        {"size": 11, "sigma": 3.0, "name": "Very Heavy Blur"}
    ]
    
    # Prepare subplot for comparison
    plt.figure(figsize=(15, 12))
    
    # Show original image
    plt.subplot(2, 3, 1)
    plt.imshow(original_image, cmap='gray')
    plt.title("Original Image")
    plt.axis('off')
    
    # Apply different blur levels
    for i, config in enumerate(blur_configs):
        print(f"\n--- {config['name']} ---")
        
        # Step 1: Create Gaussian kernel
        kernel = create_gaussian_kernel(config['size'], config['sigma'])
        
        # Step 2: Show kernel (only for small kernels)
        if config['size'] <= 7:
            print_kernel(kernel)
        
        # Step 3: Apply blur
        blurred_image = apply_convolution(original_image, kernel)
        
        # Step 4: Save result
        filename = f"blur_{config['name'].lower().replace(' ', '_')}.jpg"
        save_image_array(blurred_image, filename)
        
        # Step 5: Show in plot
        plt.subplot(2, 3, i + 2)
        plt.imshow(blurred_image, cmap='gray')
        plt.title(f"{config['name']}\nKernel: {config['size']}x{config['size']}, σ={config['sigma']}")
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig("gaussian_blur_comparison.png", dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n=== Demo Complete! ===")
    print("Key Learning Points:")
    print("1. Larger kernel size = more blur")
    print("2. Higher sigma value = stronger blur effect") 
    print("3. Convolution applies the kernel to every pixel")
    print("4. Gaussian kernels create natural-looking blur")
    print("\nFiles created:")
    print("- Individual blur results: blur_*.jpg")
    print("- Comparison chart: gaussian_blur_comparison.png")

In [7]:
def interactive_blur_experiment():
    """Let students experiment with different parameters"""
    print("\n=== Interactive Experiment ===")
    print("Try different blur parameters and see the results!")
    
    original_image = load_and_prepare_image("test.jpg")
    if original_image is None:
        return
    
    while True:
        print("\nEnter blur parameters (or 'quit' to exit):")
        
        try:
            kernel_size = input("Kernel size (3, 5, 7, 9, 11): ")
            if kernel_size.lower() == 'quit':
                break
                
            sigma = input("Sigma value (0.5-5.0): ")
            if sigma.lower() == 'quit':
                break
            
            kernel_size = int(kernel_size)
            sigma = float(sigma)
            
            # Validate inputs
            if kernel_size < 3 or kernel_size > 15 or kernel_size % 2 == 0:
                print("Kernel size must be odd number between 3 and 15")
                continue
                
            if sigma < 0.1 or sigma > 10:
                print("Sigma must be between 0.1 and 10.0")
                continue
            
            print(f"\nApplying blur: kernel={kernel_size}x{kernel_size}, sigma={sigma}")
            
            # Create and apply blur
            kernel = create_gaussian_kernel(kernel_size, sigma)
            blurred = apply_convolution(original_image, kernel)
            
            # Save and display result
            filename = f"custom_blur_{kernel_size}x{kernel_size}_sigma{sigma}.jpg"
            save_image_array(blurred, filename)
            
            # Quick display
            plt.figure(figsize=(12, 5))
            plt.subplot(1, 2, 1)
            plt.imshow(original_image, cmap='gray')
            plt.title("Original")
            plt.axis('off')
            
            plt.subplot(1, 2, 2)
            plt.imshow(blurred, cmap='gray')
            plt.title(f"Blur: {kernel_size}x{kernel_size}, σ={sigma}")
            plt.axis('off')
            
            plt.tight_layout()
            plt.show()
            
        except ValueError:
            print("Please enter valid numbers!")
        except KeyboardInterrupt:
            break
    
    print("Thanks for experimenting!")

In [8]:
demonstrate_gaussian_blur()

=== Gaussian Blur Demonstration ===
This demo shows how computer vision applies blur effects to images

Loading image: test.jpg


NameError: name 'Image' is not defined